<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## notebook adapted and extended from
(1) https://www.kaggle.com/code/sebastianji/sentiment-analysis-bert displayed under licencse Apache 2.0

(2) https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb



## executed in Google Colaboratory

In [ ]:
!python -V

Python 3.10.12


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.5 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer, AutoTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("abstracts.csv", sep ='\t')
df = df[df['Abstract'].notnull()]
df.columns = ['id','key','result']
df['id'] = df['id'].astype(int)
df.head(5)

,id,key,result
0,3301,The magnitude and timing of heat emissions fro...,False
1,20956,Purpose The purpose of this research is to con...,False
2,20984,Landscape management involves tackling both sy...,False
3,3502,"Sea level rise, one of the major consequences ...",True
4,3517,In emergency situations such as hazardous gas ...,False


In [ ]:
df.shape

(1539, 3)

In [ ]:
df = pd.read_csv("notes.csv", sep ='\t')
df = df[df['Notes'].notnull()]
df.columns = ['id','key','result']
df.head(5)

In [ ]:
df = pd.read_csv("keywords.csv", sep ='\t')
df = df[df['Keywords'].notnull()]
df.columns = ['id','key','result']
df.head(5)

In [ ]:
df = pd.read_csv("titles.csv", sep ='\t')
df = df[df['Title'].notnull()]
df.columns = ['id','key','result']
df.head(5)

In [ ]:
df = pd.read_csv("combATK.csv", sep ='\t')
df = df[df['comb'].notnull()]
df.columns = ['id','key','result']
df['id'] = df['id'].astype(int)
df.head(5)

,id,key,result
0,3301,The magnitude and timing of heat emissions fro...,False
1,20956,Purpose The purpose of this research is to con...,False
2,20984,Landscape management involves tackling both sy...,False
3,3502,"Sea level rise, one of the major consequences ...",True
4,3517,In emergency situations such as hazardous gas ...,False


In [ ]:
X_train = pd.read_csv("X_train.csv", sep ='\t')
y_train = pd.read_csv("y_train.csv", sep ='\t')
X_train['classe'] = y_train['classe']
X_train = X_train[X_train['key'].notnull()]
X_train.columns = ['key','classe']
X_train_false = X_train[X_train['classe']==False]
X_train_true = X_train[X_train['classe']==True]

X_train_true_downsampled = X_train_true.sample(X_train_false.shape[0])
X_train_true_downsampled.shape
X_train_balanced = pd.concat([X_train_true_downsampled, X_train_false])

#X_train_false_downsampled = X_train_false.sample(X_train_true.shape[0])
#X_train_false_downsampled.shape
#X_train_balanced = pd.concat([X_train_false_downsampled, X_train_true])

X_train_balanced['label']=X_train_balanced['classe'].apply(lambda x: 0 if x==False else 1)
#X_train_balanced = X_train_balanced.reset_index()
del X_train_balanced['classe']
X_train = X_train_balanced[['key']]
y_train = X_train_balanced['label'].values
print(X_train[:2])
print(y_train[:2])

                                                   key
311  Detection of urban land expansion is important...
572  Conventional CFD models can resolve complex ph...
[1 1]


In [ ]:
X_val = pd.read_csv("X_val.csv", sep ='\t')
y_val = pd.read_csv("y_val.csv", sep ='\t')
X_val['classe'] = y_val['classe']
X_val = X_val[X_val['key'].notnull()]
X_val.columns = ['key','classe']
X_val['label']=X_val['classe'].apply(lambda x: 0 if x==False else 1)
del X_val['classe']
y_val = X_val['label'].values
X_val = X_val[['key']]
print(X_val[:2])
print(y_val[:2])

                                                 key
0  Analytical models for determining the length o...
1  This paper proposes PredicTour, an approach to...
[0 1]


In [ ]:
X_train_false_downsampled.shape

NameError: ignored

In [ ]:
df=df.set_index("id")

In [ ]:
df.shape

(1539, 3)

In [ ]:
df.at[545,'result']=False
df[df.index==545]

,id,key,result
545,21083,Floods are one of hazards that periodically oc...,False


In [ ]:
#df = df[['key','result']]
df['classe'] = df['result']
del df['result']
df_false = df[df['classe']==False]
df_true = df[df['classe']==True]
df_true_downsampled = df_true.sample(df_false.shape[0])
df_true_downsampled.shape
#df_balanced = pd.concat([df_true_downsampled, df_false])
df_balanced = pd.concat([df_true, df_false])
df_balanced['label']=df_balanced['classe'].apply(lambda x: 0 if x==False else 1)
df_balanced.sample(5)

,id,key,classe,label
1164,6500,This paper maps the visual impact of mining ac...,False,0
1071,22373,"In this study, we use PGIS to evaluate and map...",True,1
900,5373,This study was conducted in South Tapanuli Dis...,False,0
899,5367,Almost every town and city in India is experie...,True,1
667,4273,Climate change and urbanization have increased...,True,1


In [ ]:
df_balanced.shape

(1539, 4)

In [ ]:
df_balanced[df_balanced.index==545]

,id,key,classe,label
545,21083,Floods are one of hazards that periodically oc...,False,0


In [ ]:
df_balanced = df_balanced.reset_index()
del df['classe']
df_balanced

,id,key,classe,label
0,22527,due turkey reliance import electricity use hyd...,1,1
1,6691,land use plan important policy instrument gove...,1,1
2,23562,present study analyze seasonal variability rel...,1,1
3,5090,research method include drone unmanned aerial ...,1,1
4,19557,disordered urban expansion encroach large amou...,1,1
...,...,...,...,...
1353,23008,purpose examine whether quality life home occu...,0,0
1354,23020,heat emission building part anthropogenic heat...,0,0
1355,7801,present construction smart city smart park chi...,0,0
1356,7803,improvement public space quality exist residen...,0,0


In [ ]:
df = df_balanced

In [ ]:
df.shape

(1539, 4)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.20,
                                                  random_state=42,
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased',
                                          do_lower_case=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('AnonymousSub/SR_bert-base-uncased',
                                          do_lower_case=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].key.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].key.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(1231, 308)

In [ ]:
label_dict = { 'False': 0, 'True': 1 }

In [ ]:
model = BertForSequenceClassification.from_pretrained("AnonymousSub/SR_bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/SR_bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels = len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=(1e-5),
                  eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    #i=0
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)
        #i+=1
        #if i==1:
          #print(batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    #print(i)
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/411 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5941896951981704
Validation loss: 0.38838641707179616
F1 Score (Weighted): 0.8506619461078665


Epoch 2:   0%|          | 0/411 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.46713794804339065
Validation loss: 0.5956885653584111
F1 Score (Weighted): 0.8040764217234806


...


Epoch 5
Training loss: 0.1279386210382362
Validation loss: 1.0204958619523874
F1 Score (Weighted): 0.8082574746126148


Epoch 6:   0%|          | 0/411 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.07805390127635971
Validation loss: 1.1203570814025634
F1 Score (Weighted): 0.8213249075318041


Epoch 7:   0%|          | 0/411 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_4.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: False
Accuracy: 127/151

Class: True
Accuracy: 130/157



In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

In [ ]:
print(classification_report(labels_flat,preds_flat))
print('Confusion Matrix:',confusion_matrix(labels_flat, preds_flat))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83       151
           1       0.84      0.83      0.84       157

    accuracy                           0.83       308
   macro avg       0.83      0.83      0.83       308
weighted avg       0.83      0.83      0.83       308

Confusion Matrix: [[127  24]
 [ 27 130]]


ALL PAPERS

In [ ]:
X_fest=pd.read_csv("ALL PAPERS MAY10.csv", sep ='\t')
X_fest.columns = ['Record Number','key']

In [ ]:
y_fest = X_fest * 0
y_fest.columns = ['classe','dummy']
y_fest['classe'].values

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_fest = X_fest['value']
y_fest=y_fest.apply(lambda x: 0 if x==False else 1)
y_fest.values

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    X_fest['key'].values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
#labels_test = torch.tensor(y_test['classe'].values)
labels_test = torch.tensor(y_fest['classe'].values)

In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
model.load_state_dict(torch.load('atkb_finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_test = DataLoader(dataset_test,
                                   sampler=SequentialSampler(dataset_test),
                                   batch_size=batch_size)

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

In [ ]:
X_test['prediction']=preds_flat
X_test.to_csv('predtctions.csv', index = False, sep='\t')

In [ ]:
X_test

,Record Number,comb,prediction
0,1,Recent advances in information and communicati...,1
1,2,This paper proposes a cause to reflect on adap...,0
2,3,"In recent years, there has been a renewed inte...",1
3,4,This book studies the increasing use of data a...,0
4,5,Digital Participatory Planning outlines develo...,1
...,...,...,...
35791,53110,The article reports on several technological i...,0
35792,53111,The author offers forecasts related to the dem...,0
35793,53112,The article discusses research being done whic...,0
35794,53113,"SORENSEN--Lena. Lena Sorensen, born December 8...",0


DATASET TEST Jan3

In [ ]:
#X_test=pd.read_csv("Jan3ATK.csv", sep ='\t')
X_fest=pd.read_csv("abstractsJan3BD.csv", sep ='\t')
X_fest.columns = ['Record Number','key','value']
#y_val=pd.read_csv("y_val.csv", sep ='\t')

In [ ]:
X_test

,Record Number,key,value
0,178,Cities today exhibit three key characteristics...,True
1,179,Rising levels of air contamination are dictati...,True
2,180,Mean radiant temperature is a key element in e...,True
3,181,The importance of multi-sensory perception in ...,False
4,182,To ensure a reliable operation over the life t...,False
...,...,...,...
171,352,Light design as a prominent and self-sufficien...,False
172,353,Building change detection from remote sensing ...,True
173,354,Maintaining suitable temperature level around ...,False
174,355,Saudi Arabia is affected by various types of n...,True


In [ ]:
X_test.shape

(3549, 2)

In [ ]:
y_test = X_test * 0
y_test.columns = ['classe','dummy']
y_test['classe'].values

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_fest = X_fest['value']
y_fest=y_fest.apply(lambda x: 0 if x==False else 1)
y_fest.values

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1])

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    X_fest['key'].values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
#labels_test = torch.tensor(y_test['classe'].values)
labels_test = torch.tensor(y_fest.values)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
#model_name='atk_rs42_828_finetuned_BERT_epoch_4.model'
#model_name='atk_rs42_82_finetuned_BERT_epoch_7.model'
#model_name='atk_rs42_85v2_lr1_finetuned_BERT_epoch_4.model'
#model_name='atk_rs42_86v2lr1_finetuned_BERT_epoch_2.model'
#model_name='atk_rs42_lr_1_finetuned_BERT_epoch_3.model'
#model_name='best_ATK_86_finetuned_BERT_epoch_5.model'
model_name='good_atk_85_finetuned_BERT_epoch_3.model'

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/models/'+model_name, map_location=torch.device('cuda')))


<All keys matched successfully>

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_test = DataLoader(dataset_test,
                                   sampler=SequentialSampler(dataset_test),
                                   batch_size=batch_size)

In [ ]:
import random
import numpy as np

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: False
Accuracy: 54/80

Class: True
Accuracy: 71/90



In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

In [ ]:
X_test['prediction']=preds_flat
X_test.to_csv(model_name+'.csv', index = False, sep='\t')

NameError: ignored

In [ ]:
X_test2=pd.read_csv(model_name+'.csv', sep ='\t')
X_test2

,Record Number,key,prediction
0,1,Sufficient research on cultural ecosystem serv...,0
1,2,Habituation and sensitization represent nonass...,0
2,3,The rapid change and transformation of socioec...,0
3,5,"Trawlers are the key fishing fleet, which is c...",1
4,6,Nonlinearly constrained nonlinear programming ...,0
...,...,...,...
3544,3564,Urban planning has always reflected the major ...,0
3545,3565,This research aims to define a low-cost replic...,1
3546,3566,The urban heat island (UHI) effect caused by u...,1
3547,3567,Municipalities are responsible for organizing ...,0


In [ ]:
print(classification_report(labels_flat,preds_flat))
print('Confusion Matrix:',confusion_matrix(labels_flat, preds_flat))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82        99
           1       0.77      0.75      0.76        77

    accuracy                           0.80       176
   macro avg       0.79      0.79      0.79       176
weighted avg       0.80      0.80      0.80       176

Confusion Matrix: [[82 17]
 [19 58]]


In [ ]:
df[df.data_type=='train'].index.values

array([  442,  1116,   277, ...,  1383,  1469, 11816])

TESTOWANIE ZESTAWÓW PODZIELONYCH PRZEZ W2V

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    X_train.key.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val.key.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
label_dict = { 'False': 0, 'True': 1 }

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=(1e-5)*1,
                  eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    #i=0
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)
        #i+=1
        #if i==1:
          #print(batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    #print(i)
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/360 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5851683942187164
Validation loss: 0.47784242861800724
F1 Score (Weighted): 0.7525000000000002


Epoch 2:   0%|          | 0/360 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.42951326333245055
Validation loss: 0.7233162879271225
F1 Score (Weighted): 0.8061201944321698


...

Epoch 4:   0%|          | 0/360 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.1720933412680299
Validation loss: 0.8297758960061603
F1 Score (Weighted): 0.8222124670763828


Epoch 5:   0%|          | 0/360 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.10454918502897878


KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: False
Accuracy: 98/126

Class: True
Accuracy: 122/144



In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

In [ ]:
print(classification_report(labels_flat,preds_flat))
print('Confusion Matrix:',confusion_matrix(labels_flat, preds_flat))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80       126
           1       0.81      0.85      0.83       144

    accuracy                           0.81       270
   macro avg       0.81      0.81      0.81       270
weighted avg       0.81      0.81      0.81       270

Confusion Matrix: [[ 98  28]
 [ 22 122]]
